In [1]:
import zipfile
import os
import glob
import shutil

In [2]:
data_zip = '/content/Brain_tumor.zip'
data_dir = '/content/data/'
zip_file = zipfile.ZipFile(data_zip,'r')
zip_file.extractall(data_dir)

In [3]:
os.mkdir(os.path.join(data_dir,'train'))
os.mkdir(os.path.join(data_dir,'val'))

In [4]:
data_train = '/content/data/train'
data_train_no = os.path.join(data_train,'no')
os.mkdir(data_train_no)
data_train_yes = os.path.join(data_train,'yes')
os.mkdir(data_train_yes)
data_val = '/content/data/val'
data_val_no = os.path.join(data_val,'no')
os.mkdir(data_val_no)
data_val_yes = os.path.join(data_val,'yes')
os.mkdir(data_val_yes)

In [5]:
split = 0.8

no_size = len(os.listdir('/content/data/brain_tumor_dataset/no'))
yes_size = len(os.listdir('/content/data/brain_tumor_dataset/yes'))

for i,img in enumerate(glob.glob("/content/data/brain_tumor_dataset/no/*")):
  if(i < no_size*split):
    print(i,img)
    shutil.move(img,data_train_no)
  else:
    shutil.move(img,data_val_no)

for i,img in enumerate(glob.glob("/content/data/brain_tumor_dataset/yes/*")):
  if(i < yes_size*split):
    print(i,img)
    shutil.move(img,data_train_yes)
  else:
    shutil.move(img,data_val_yes)

0 /content/data/brain_tumor_dataset/no/no 90.jpg
1 /content/data/brain_tumor_dataset/no/36 no.jpg
2 /content/data/brain_tumor_dataset/no/1 no.jpeg
3 /content/data/brain_tumor_dataset/no/no 97.jpg
4 /content/data/brain_tumor_dataset/no/32 no.jpg
5 /content/data/brain_tumor_dataset/no/no 7.jpeg
6 /content/data/brain_tumor_dataset/no/No21.jpg
7 /content/data/brain_tumor_dataset/no/14 no.jpg
8 /content/data/brain_tumor_dataset/no/no 9.png
9 /content/data/brain_tumor_dataset/no/N22.JPG
10 /content/data/brain_tumor_dataset/no/6 no.jpg
11 /content/data/brain_tumor_dataset/no/No15.jpg
12 /content/data/brain_tumor_dataset/no/no 2.jpg
13 /content/data/brain_tumor_dataset/no/29 no.jpg
14 /content/data/brain_tumor_dataset/no/N19.JPG
15 /content/data/brain_tumor_dataset/no/no 6.jpg
16 /content/data/brain_tumor_dataset/no/No16.jpg
17 /content/data/brain_tumor_dataset/no/N5.jpg
18 /content/data/brain_tumor_dataset/no/9 no.jpg
19 /content/data/brain_tumor_dataset/no/No12.jpg
20 /content/data/brain_tum

In [55]:
import torch
import torchvision
from torchvision import datasets,transforms

In [56]:
train_transforms = transforms.Compose([transforms.Resize((224,224)),
                                       transforms.ToTensor(),
                                       torchvision.transforms.Normalize(
                                           mean=[0.485, 0.456, 0.406],
                                           std=[0.229, 0.224, 0.225],),
                                       ])
test_transforms = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.ToTensor(),
                                      torchvision.transforms.Normalize(
                                          mean=[0.485, 0.456, 0.406],
                                          std=[0.229, 0.224, 0.225],),
                                      ])

In [57]:
train_data = datasets.ImageFolder(data_train,transform = train_transforms)
test_data = datasets.ImageFolder(data_val,transform = test_transforms)

train_loader = torch.utils.data.DataLoader(train_data,shuffle=True,batch_size =16)
test_loader = torch.utils.data.DataLoader(test_data,shuffle=True,batch_size =16)


In [58]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [59]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 32, 5)
        self.fc1 = nn.Linear(32 * 24 * 24, 2048)
        self.fc2 = nn.Linear(2048, 512)
        self.fc3 = nn.Linear(512, 2)

    def forward(self, x):
        #print(x.shape)
        x = self.pool(F.relu(self.conv1(x)))
        #print("conv1:", x.shape)
        x = self.pool(F.relu(self.conv2(x)))
        #print("conv2:", x.shape)
        x = self.pool(F.relu(self.conv3(x)))
        #print("conv3:", x.shape)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        #print("fc1:", x.shape)
        x = F.relu(self.fc2(x))
        #print("fc2:", x.shape)
        x = self.fc3(x)
        #print("fc3:", x.shape)
        return x
net = Net()
net = net.to(device)

In [60]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [61]:
for epoch in range(30):  # loop over the dataset multiple times
    print("Epoch",epoch)
    train_loss = 0.0
    val_loss = 0.0
    correct = 0
    net.train()
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        #print(inputs.size(0))
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        #print("loss",loss)
        # print statistics
        train_loss += loss.item()*(inputs.size(0))
    train_loss = train_loss/len(train_loader.dataset)
    print("train_loss",train_loss)
    #print('Finished Training')

    net.eval()
    for i, data1 in enumerate(test_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data1
        #print(inputs.size(0))
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        val_loss += loss.item()*(inputs.size(0))
        _, pred = torch.max(outputs, 1)
        correct += (pred == labels).sum()
    val_loss = val_loss/len(test_loader.dataset)
    print("val_loss",val_loss)
    acc = correct/len(test_loader.dataset)
    print('acc',acc.item()*100)

Epoch 0
train_loss 0.6904315320141797
val_loss 0.6849471449851989
acc 62.00000047683716
Epoch 1
train_loss 0.6806115223269157
val_loss 0.6717254781723022
acc 62.00000047683716
Epoch 2
train_loss 0.667891009981409
val_loss 0.6604048490524292
acc 62.00000047683716
Epoch 3
train_loss 0.6554083327941707
val_loss 0.6484234523773194
acc 62.00000047683716
Epoch 4
train_loss 0.6411356823197727
val_loss 0.6330976867675782
acc 62.00000047683716
Epoch 5
train_loss 0.6223245034076897
val_loss 0.6135305070877075
acc 62.00000047683716
Epoch 6
train_loss 0.5994229143476252
val_loss 0.5910801887512207
acc 62.00000047683716
Epoch 7
train_loss 0.5723595014346644
val_loss 0.5667279517650604
acc 69.9999988079071
Epoch 8
train_loss 0.5453040156458399
val_loss 0.5434498071670533
acc 74.00000095367432
Epoch 9
train_loss 0.5174180007920477
val_loss 0.523146014213562
acc 77.99999713897705
Epoch 10
train_loss 0.49154440596185883
val_loss 0.5176272702217102
acc 79.99999523162842
Epoch 11
train_loss 0.46625803902

In [62]:
torch.save(net.state_dict(),'/content/data/model_weigths.pt')

In [63]:
net.load_state_dict(torch.load('/content/data/model_weigths.pt'))
net.eval()

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=18432, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=2, bias=True)
)

In [65]:
from PIL import Image
test_image = '/content/data/val/yes/Y101.jpg'
test_image1 = '/content/data/val/no/17 no.jpg'
test_img = Image.open(test_image)
print(test_img)
transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.Resize((224, 224)),  # adjust to model's expected size
    transforms.ToTensor(),
])
img_test = transform(test_img).unsqueeze(0)  # add batch dimension
img_test = img_test.to(device)
# Step 4: Run inference
with torch.no_grad():
    output = net(img_test)
    print(output)
    _, prediction = torch.max(output,1)
    print(prediction)

print(f'Predicted class: {int(prediction)}')  # 0 or 1


<PIL.JpegImagePlugin.JpegImageFile image mode=L size=300x336 at 0x7E4BF14046D0>
tensor([[-0.4826,  0.4812]], device='cuda:0')
tensor([1], device='cuda:0')
Predicted class: 1
